In [1]:
from pathlib import Path
import typing
import json

In [2]:
path_to_savegame = Path('C:/Users/Victor/Documents/Paradox Interactive/Crusader Kings II/save games')
save_game_name = '771_01_23.ck2'
complete_path = path_to_savegame / save_game_name

In [3]:
class InfoRepresentation(typing.Protocol):
    
    @staticmethod
    def create(raw_into : str) -> typing.Self:
        pass
    
    def to_raw_string(self) -> str:
        pass
    @staticmethod
    def corresponds(raw_info:str) ->bool:
        pass
    def change_value(self , other : typing.Any) -> None:
        pass

In [10]:
class OneLineKeyValueInfo:
    def __init__(self , 
                 data_key : str ,
                 data_value : str , 
                 start_spaces : int ,
                 end_spaces : int):
        self.data_key = data_key
        self.data_value = data_value
        self.start_spaces= start_spaces
        self.end_spaces = end_spaces
    @staticmethod
    def create(raw_info : str) -> typing.Self:
        start_spaces = raw_info.count(r'\t')
        end_spaces = raw_info.count(r'\n')
        try :
            data_name , data_value = raw_info.removeprefix(r'\t').removesuffix(r'\n').split('=')
        except :
            print(raw_info)
        return OneLineKeyValueInfo(
            data_key = data_name,
            data_value = data_value,
            start_spaces = start_spaces, 
            end_spaces= end_spaces
        )
    
    @staticmethod
    def corresponds(raw_info : str) -> bool:
        
        return '=' in raw_info and not raw_info.endswith(r'=\n') and "{" not in raw_info
    
    def to_raw_string(self) -> str:
        return r"\t"*self.start_spaces + f'{self.data_key}={self.data_value}' + r"\n"*self.end_spaces
    
    def change_value(self , value : str) :
        self.data_value = value

class OneLineListInfo:
    def __init__(self , info_list : list[str],start_spaces : int , end_spaces : int):
        self.info_list = info_list
        self.start_spaces = start_spaces
        self.end_spaces = end_spaces
    
    @staticmethod
    def create(raw_info : str) -> typing.Self:
        
        start_spaces = raw_info.count(r'\t')
        end_spaces = raw_info.count(r'\n')
        
        data = raw_info.removeprefix(r'\t').removesuffix(r'\n').split(' ')
        
        return OneLineListInfo(info_list = data,
                               start_spaces = start_spaces,
                               end_spaces = end_spaces
                               )
    @staticmethod
    def corresponds(raw_info : str) -> bool:
        
        return '=' not in raw_info and raw_info.count(' ') > 1
    
    def to_raw_string(self) -> str:
        return r'\t' * self.start_spaces + ' '.join(self.info_list) + r'\n' * self.end_spaces
    
    def change_value(self , other_list : list[str]):
        self.info_list = other_list

class OneLineKeyListInfo:
    
    def __init__(self , data_key : str , data_list : OneLineListInfo,start_spaces : int , end_spaces : int):
        
        self.data_key = data_key
        self.data_list = data_list
        self.start_spaces = start_spaces
        self.end_spaces = end_spaces
    
    def create(raw_info : str) -> typing.Self:
        
        start_spaces = raw_info.count(r'\t')
        end_spaces = raw_info.count(r'\n')
        
        data_key , data_list_str = raw_info.removeprefix(r'\t').removesuffix(r'\n').split('=')
        data_list = OneLineListInfo.create(
                    raw_info = data_list_str.replace('{',r'\t').replace('}',r'\n')
        )
        return OneLineKeyListInfo(
            data_key = data_key ,
            data_list = data_list,
            start_spaces=start_spaces,
            end_spaces=end_spaces
        )
    
    @staticmethod
    def corresponds(raw_info : str) -> bool:
        return '=' in raw_info and '{' in raw_info
    def to_raw_string(self) -> str:
        data_list_str = self.data_list.to_raw_string().replace(r'\t','{').replace(r'\n','}')
        
        return r"\t"*self.start_spaces + f'{self.data_key}={data_list_str}' + r"\n"*self.end_spaces
class DictInfo:
    
    def __init__(self , 
                 key : str ,
                 value : list[typing.Self | OneLineKeyListInfo | OneLineListInfo | OneLineKeyValueInfo],
                 start_spaces : int,
                 end_spaces : int
                 ):
        
        self.key = key
        self.value = value
        self.start_spaces = start_spaces
        self.end_spaces = end_spaces
    
    @staticmethod
    def corresponds(raw_info : str):
        
        return raw_info.endswith(r'=\n')

    @staticmethod
    def create( raw_key_data :str , ck2generator : typing.Generator[str,None,None] ) -> typing.Self:
        
        start_spaces = raw_key_data.count(r'\t')
        end_spaces = raw_key_data.count(r'\n')
        
        key_data = raw_key_data.replace(r'\n','').replace(r'\t','').replace('=','')
        
        dict_data = []
        
        open_parenthesis = next(ck2generator)
        if not open_parenthesis.endswith(r'{\n'):
            raise ValueError(f'Expected a curly bracket but only got {open_parenthesis}')
        
        for line in ck2generator:
            
            if OneLineKeyValueInfo.corresponds(raw_info = line):
                
                new_value = OneLineKeyValueInfo.create(raw_info = line)
                dict_data.append(new_value)
            
            if OneLineListInfo.corresponds(raw_info = line):
                
                new_value = OneLineListInfo.create(raw_info = line)
                dict_data.append(new_value)
            
            if OneLineKeyListInfo.corresponds(raw_info = line):
                
                new_value = OneLineKeyListInfo.create(raw_info = line)
                dict_data.append(new_value)
            
            if DictInfo.corresponds(raw_info = line):
                
                new_value = DictInfo.create(raw_key_data = line , ck2generator = ck2generator)
                dict_data.append(new_value)
            
            if line.endswith(r'}\n'):
                return DictInfo(
                    key = key_data ,
                    value = dict_data,
                    start_spaces= start_spaces,
                    end_spaces = end_spaces
                )
        raise Exception('something is wrong !')

In [11]:
class SaveFileParser:
    
    def __init__(self , file_path : Path ):
        self.path = file_path
        self.generator = SaveFileParser.read_file_line_by_line(file_path = self.path)
    @staticmethod
    def read_file_line_by_line(file_path : Path):
        with open(file_path, 'r') as file:
            for line in file:
                yield line
    
    def one_line_data(self , line) -> bool:
        return '"' in line
    
    def parse_data(self):
        
        data = []
        first_line = next(self.generator) 
        
        for line in self.generator:
        
            if OneLineKeyValueInfo.corresponds(raw_info = line):
                    
                new_value = OneLineKeyValueInfo.create(raw_info = line)
                data.append(new_value)
            
            if OneLineListInfo.corresponds(raw_info = line):
                
                new_value = OneLineListInfo.create(raw_info = line)
                data.append(new_value)
            
            if OneLineKeyListInfo.corresponds(raw_info = line):
                
                new_value = OneLineKeyListInfo.create(raw_info = line)
                data.append(new_value)
            
            if DictInfo.corresponds(raw_info = line):
                
                new_value = DictInfo.create(raw_key_data = line , ck2generator = self.generator)
                data.append(new_value)
        
        return data

In [12]:
line = '\tlastmonthincometable={0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.10000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000}\n'
line_info = OneLineKeyListInfo.create(raw_info=line)


In [13]:
game_generator = SaveFileParser(file_path=complete_path)

In [14]:
d = game_generator.parse_data()

				1060322="antagonizing_target_key" 1060547="swaying_target_key" 



UnboundLocalError: cannot access local variable 'data_name' where it is not associated with a value

In [ ]:
d

In [ ]:
next(game_generator)

'\tdate="771.1.23"\n'

In [ ]:
next(game_generator)

'\tplayer=\n'

In [ ]:
next(game_generator)

'\t{\n'

In [ ]:
next(game_generator)

'\t\tid=1068759\n'

In [ ]:
next(game_generator)

'\t\ttype=66\n'

In [ ]:
next(game_generator)

'\t}\n'

In [ ]:
next(game_generator)

'\tplayer_realm="c_olvia"\n'